## Qualitative Results Auto View

Running by this ipynb you will get the full qualitative results of the paper.

The camera position is already in the dataset, check the dataset folder for more information.

Remember to install the package you need:

`conda install -c conda-forge pyvista ipywidgets Pillow`

Some package issues I met before:
- https://github.com/slundberg/shap/issues/1043
- memory leak problem: https://github.com/pyvista/pyvista/issues/482

In [ ]:
# Created: 2023-05-11 22:35
# @Copyright (C) 2023-now, RPL, KTH Royal Institute of Technology
# @Author: Kin ZHANG  (https://kin-zhang.github.io/)

# If you find this repo helpful, please cite the respective publication in DUFOMap.
# This script is licensed under the terms of the MIT license.
# For a copy, see <https://opensource.org/licenses/MIT>.

import numpy as np
import pyvista as pv
pv.set_jupyter_backend('trame')

def view_pcd2img(pcd_np_data, camera_view_file, save_img_path, show_pred_dynamic=False, show_false_diff=False):
    
    label_colors = {0: [145, 191, 219], # blue static TN
                    1: [239, 133, 84],   # orange FN
                    2: [255, 255, 191], # yellow TP
                    3: [239, 133, 84]   # red [240, 59, 32] FP, orange [239, 133, 84]
                    }
    plotter = pv.Plotter(off_screen=True, notebook=True)
    plotter.enable_eye_dome_lighting()
    plotter.enable_trackball_style()
    plotter.set_background([236, 235, 235])
    # if show_dynamic:
    cloud = pv.PolyData(pcd_np_data[:,:3])
    labels = pcd_np_data[:,3]
    show_label = [0,1,2,3]
    
    if show_pred_dynamic and not show_false_diff:
        labels[labels==3] = 0
        labels[labels==1] = 2
        show_label = [0,2]
    elif show_pred_dynamic and show_false_diff: # show output static
        labels[labels==3] = 0
        show_label = [0,1,2]
    elif not show_pred_dynamic and show_false_diff:
        show_label = [0,3]
        print("Show Static Map with False Predicted")
    elif not show_pred_dynamic and not show_false_diff: # show output static with single color (blue)
        labels[labels==3] = 0
        show_label = [0]

    cloud["labels"] = labels.flatten()

    for label, color in label_colors.items():
        if label not in show_label:
            continue
        draw_pt = cloud.extract_points(cloud["labels"] == label)
        if draw_pt.n_points>0:
            plotter.add_mesh(draw_pt, point_size=5, color=color)
    
    loaded_camera_position = []
    with open(camera_view_file, "r") as file:
        lines = file.readlines()
        for line in lines:
            loaded_camera_position.append(eval(line.strip()))
            
    loaded_camera_position = tuple(loaded_camera_position)
    plotter.camera_position = loaded_camera_position
    plotter.show(screenshot=save_img_path)
    plotter.close()
    plotter.clear()
    plotter.close()

In [ ]:
# Please Open this
import sys, os
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '../..' ))
sys.path.insert(0, BASE_DIR)
from utils.pcdpy3 import load_pcd
from utils import check_file_exists

Result_Folder = "/home/kin/data/Dynamic_Papers_assets/BeautyMap" 
algorithms = ["octomap", "dynablox", "erasor", "edomap"]
data_name = "05"
camera_view = "camera_position_close.txt"

os.makedirs(os.path.join(Result_Folder, "imgs"), exist_ok=True)

In [ ]:
save_type = "dynamic"
gt_pcd_path = f"{Result_Folder}/{data_name}/gt_cloud.pcd"
check_file_exists(gt_pcd_path)
gt_pc_ = load_pcd(gt_pcd_path)
img_path = f"{Result_Folder}/{data_name}/imgs/gt_{save_type}.png"
if not os.path.exists(img_path):
    gt_pc_.np_data[gt_pc_.np_data[:,3] == 1, 3] = 2
    view_pcd2img(gt_pc_.np_data, camera_view, img_path, show_pred_dynamic=True, show_false_diff=False)
    print(f"Saved {img_path} on ground truth")
for algo in algorithms:
    et_pcd_path = f"{Result_Folder}/{data_name}/eval/{algo}_output_exportGT.pcd"
    check_file_exists(et_pcd_path)
    et_pc_ = load_pcd(et_pcd_path)
    TN_mask = (et_pc_.np_data[:,3] == 0) * (gt_pc_.np_data[:,3] == 0)
    FN_mask = (et_pc_.np_data[:,3] == 1) * (gt_pc_.np_data[:,3] == 0)
    TP_mask = (et_pc_.np_data[:,3] == 1) * (gt_pc_.np_data[:,3] == 1)
    FP_mask = (et_pc_.np_data[:,3] == 0) * (gt_pc_.np_data[:,3] == 1)
    et_pc_.np_data[TN_mask, 3]=0 # TN
    et_pc_.np_data[FN_mask, 3]=1 # FN
    et_pc_.np_data[TP_mask, 3]=2 # TP
    et_pc_.np_data[FP_mask, 3]=3 # FP
    
    img_path = f"{Result_Folder}/{data_name}/imgs/{algo}_{save_type}.png"
    view_pcd2img(et_pc_.np_data, camera_view, img_path, show_pred_dynamic=True, show_false_diff=False)
    print(f"Saved {img_path}")

In [ ]:
save_type = "static"
gt_pcd_path = f"{Result_Folder}/{data_name}/gt_cloud.pcd"
check_file_exists(gt_pcd_path)
gt_pc_ = load_pcd(gt_pcd_path)
img_path = f"{Result_Folder}/{data_name}/imgs/gt_{save_type}.png"
if not os.path.exists(img_path):
    gt_pc_.np_data[gt_pc_.np_data[:,3] == 1, 3] = 2
    view_pcd2img(gt_pc_.np_data, camera_view, img_path, show_pred_dynamic=False, show_false_diff=True)
    print(f"Saved {img_path} on ground truth")
for algo in algorithms:
    et_pcd_path = f"{Result_Folder}/{data_name}/eval/{algo}_output_exportGT.pcd"
    check_file_exists(et_pcd_path)
    et_pc_ = load_pcd(et_pcd_path)
    TN_mask = (et_pc_.np_data[:,3] == 0) * (gt_pc_.np_data[:,3] == 0)
    FN_mask = (et_pc_.np_data[:,3] == 1) * (gt_pc_.np_data[:,3] == 0)
    TP_mask = (et_pc_.np_data[:,3] == 1) * (gt_pc_.np_data[:,3] == 1)
    FP_mask = (et_pc_.np_data[:,3] == 0) * (gt_pc_.np_data[:,3] == 1)
    et_pc_.np_data[TN_mask, 3]=0 # TN
    et_pc_.np_data[FN_mask, 3]=1 # FN
    et_pc_.np_data[TP_mask, 3]=2 # TP
    et_pc_.np_data[FP_mask, 3]=3 # FP
    
    img_path = f"{Result_Folder}/{data_name}/imgs/{algo}_{save_type}.png"
    view_pcd2img(et_pc_.np_data, camera_view, img_path, show_pred_dynamic=False, show_false_diff=True)
    print(f"Saved {img_path}")
    # break

Combine all img together to one big figure

In [ ]:
from PIL import Image
from IPython.display import display

def read_all_img(show_type="clean", data_name="05"):
    image_filenames = []
    image_filenames.append(f"{Result_Folder}/{data_name}/imgs/gt_{show_type}.png")
    for algo in algorithms:
        image_filenames.append(f"{Result_Folder}/{data_name}/imgs/{algo}_{show_type}.png")
    images = [Image.open(filename) for filename in image_filenames]
    return images

def show_combined_img(images_clean, images_dynam, data_name="05"):
    column_space = 5
    row_space = 5

    # Find the max height and total width for the new combined image
    max_height_row1 = max(img.height for img in images_clean)
    max_height_row2 = max(img.height for img in images_dynam)
    total_height = max_height_row1 + max_height_row2 + row_space
    total_width = max(
        sum(img.width for img in images_clean) + column_space * (len(images_clean) - 1),
        sum(img.width for img in images_dynam) + column_space * (len(images_dynam) - 1),
    )

    # Create a new image with the combined dimensions and a white background
    combined_image = Image.new("RGB", (total_width, total_height), color="white")


    x_offset = 0
    for img in images_dynam:
        combined_image.paste(img, (x_offset, 0))
        x_offset += img.width + column_space
    x_offset = 0
    for img in images_clean:
        combined_image.paste(img, (x_offset, max_height_row1 + row_space))
        x_offset += img.width + column_space


    # Save the combined image
    combined_image.save(f"{Result_Folder}/{data_name}/imgs/total.png")
    display(combined_image)

def show_three_combined_img(images_full, images_clean, images_dynam, data_name="05"):
    column_space = 5
    row_space = 5

    # Find the max height and total width for the new combined image
    max_height_row0 = max(img.height for img in images_full)
    max_height_row1 = max(img.height for img in images_clean)
    max_height_row2 = max(img.height for img in images_dynam)
    total_height = max_height_row0 + max_height_row1 + max_height_row2 + 2*row_space

    total_width = max(
        sum(img.width for img in images_full) + column_space * (len(images_full) - 1),
        sum(img.width for img in images_clean) + column_space * (len(images_clean) - 1),
        sum(img.width for img in images_dynam) + column_space * (len(images_dynam) - 1),
    )

    # Create a new image with the combined dimensions and a white background
    combined_image = Image.new("RGB", (total_width, total_height), color="white")

    # Paste images_full row
    x_offset = 0
    for img in images_full:
        combined_image.paste(img, (x_offset, 0))
        x_offset += img.width + column_space

    # Paste images_dynam row
    x_offset = 0
    for img in images_dynam:
        combined_image.paste(img, (x_offset, max_height_row0 + row_space))
        x_offset += img.width + column_space

    # Paste images_clean row
    x_offset = 0
    for img in images_clean:
        combined_image.paste(img, (x_offset, max_height_row0 + max_height_row1 + 2*row_space))
        x_offset += img.width + column_space

    # Save the combined image
    combined_image.save(f"{Result_Folder}/{data_name}/imgs/total.png")
    display(combined_image)


print("Showing images for data: ", data_name, f"saving {Result_Folder}/{data_name}/imgs/total.png")
# images_full = read_all_img(show_type="full", data_name=select_data)
images_clean = read_all_img(show_type="static", data_name=data_name)
images_dynam = read_all_img(show_type="dynamic", data_name=data_name)
show_combined_img(images_clean, images_dynam, data_name=data_name)
# show_three_combined_img(images_full, images_dynam,images_clean, data_name=select_data)